## Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import os

In [ ]:
#!pip install dask

## Extract data from gcp

In [85]:
#this does not group by tariff type
def process_group_data(g):
    #not required for Chris
    df=pd.read_csv(f"https://storage.googleapis.com/energy_usage_prediction_903/df_{g}_v1.csv")
    df.drop(columns='Unnamed: 0',inplace=True)
    #imputing zeros and nans on group avverage as opposed to per house for speed
    df1=df.loc[:,['LCLid','Acorn_Group','DateTime','KWH/hh','Tariff']]
    df1.loc[:,'DateTime']=pd.to_datetime(df1['DateTime'])
    #group average
    df2=df1.groupby(by=[df.DateTime,'Tariff']).mean()
    df2.sort_index(inplace=True)
    #fill 0s with nans
    df2['KWH/hh'].replace(0,np.nan,inplace=True)
    #fill nans for both existing ones and any created due to missing HH 
    date_range = pd.DataFrame(pd.date_range(df2.index[0],df2.index[-1], freq='30 min'),columns=['DateTime'])
    df2.reset_index(inplace=True)
    df2.loc[:,'DateTime']=pd.to_datetime(df2['DateTime'])
    df3=date_range.merge(df2,on='DateTime',how='outer')
    if np.sum(df3['KWH/hh'].isna())!=0:
        df3.fillna(method='bfill',inplace=True)
    return df3

## Check start and end date for each group

In [ ]:
groups=['A','B','C','D','F','G','H','I','J','K','L','M','N','O','P','Q']
def extract_start_end_time_bygroup(groups):
    g_dict={}
    g_dict['E']=[['2011-11-24 11:00:00', '2014-02-28 00:00:00']]
    for g in groups:
        df=pd.read_csv(f"https://storage.googleapis.com/energy_usage_prediction_903/df_{g}_v1.csv",usecols=['DateTime','KWH/hh'])
        df.set_index('DateTime',inplace=True)
        df_av=df.groupby(by=df.index).mean()
        df_av.sort_index(inplace=True)
        g_dict[g]=[[df_av.index[0],df_av.index[-1]]]
        
    return g_dict

In [ ]:
# df=pd.read_csv('https://storage.googleapis.com/energy_usage_prediction_903/df_E_v1.csv',usecols=['DateTime','KWH/hh'])
# df.set_index('DateTime',inplace=True)
# t=[df.index[0],df.index[-1]]#['2012-07-09 11:30:00', '2014-02-28 00:00:00']

In [ ]:
# #read data in chunks of 1 million rows at a time
# chunk = pd.read_csv('https://storage.googleapis.com/energy_usage_prediction_903/df_E_v1.csv',chunksize=1000000)
# pd_df = pd.concat(chunk)

## merge_all_data

In [ ]:
groups=['A','B','C','D','F','G','H','I','J','K','L','M','N','O','P','Q']
def merge_groups(groups,use_tariff=True):
    for g  in groups:
        if use_tariff==True:
            tariffs=['Std','ToU']
            for tr in tariffs:
                df2=pd.read_csv(f"https://storage.googleapis.com/energy_usage_prediction_903/df_{g}_avg_{tr}_v1.csv").drop(columns='Unnamed: 0')
                df2['Acorn_Group']=g
                df=pd.concat([df, df2])                
        else:
            df1=pd.read_csv(f"https://storage.googleapis.com/energy_usage_prediction_903/df_{g}_avg_v1.csv").drop(columns='Unnamed: 0')
            df1['Acorn_Group']=g
            df=pd.concat([df, df1])
    return df
    

## Save files

In [ ]:
df_all_avg_tariff_v1=merge_groups(groups,use_tariff=True)
df_all_avg_v1=merge_groups(groups,use_tariff=False)

path=os.path.join(os.getcwd(),"../Team_Energy/data/")
print(path)
df_all_avg_tariff_v1.to_csv(path+"df_all_avg_tariff_v1.csv",header=True,index=False)
df_all_avg_v1.to_csv(path+"df_all_avg_v1.csv",header=True,index=False)